# SEQUENCE-TO-SEQUENCE NMT WITH ATTENTION MECHANISM

# DATA

In [1]:
import numpy as np
import re
import string
import unicodedata
from unicodedata import normalize
import sys
import indicnlp
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
# from keras.models import Model
# from keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.utils import plot_model
import numpy as np
# !pip install tensorflow-addons==0.11.2
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

from tensorflow import keras
# !pip install indic-nlp-library : this is necessary, without this common wont be found and correct indic nlp wont get installed
import os
import io
import time
INDIC_NLP_LIB_HOME=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\anoopkunchukuttan-indic_nlp_library-eccde81"
INDIC_NLP_RESOURCES=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

# LOAD DATA

In [2]:
def load_doc(doc):
    file = open(doc , mode = 'rt' , encoding = 'utf -8')
    text = file.read()
    file.close()
    return text

def clean_english_data(lines):
    re_print = re.compile('[^%s]' %re.escape(string.printable))
    table = str.maketrans('' , '' , string.punctuation)
    
    
    clean_eng_lines = []
    ans = list()
    ans.append(lines)
    
    for line in ans:
        line = normalize('NFD' , line).encode('ascii' , 'ignore')
        line = line.decode('UTF-8')
        line = line.split()
        line = [word.lower() for word in line]
        line = [word.translate(table) for word in line]
        line = [word for word in line if word.isalpha()]
        line = ' '.join(line)
        line = '<start> ' + line +' <end>'
        clean_eng_lines.append(line)
    return clean_eng_lines

def clean_text(line):
    text = line
    text = text.replace(u',' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u'(' ,'')
    text = text.replace(u')' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u':' ,'')
    text = text.replace(u"'" ,'')
    text = text.replace(u"'" ,'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text = re.sub('[a-zA-Z]' , '' , text)
    text = re.sub('[0-9+\-*/.%&!]' , '' , text)
    return text

def clean_hindi_data(lines):
    clean_hin_lines = []
    ans = list()
    ans.append(lines)
    for line in ans:
        remove_nuktas = True
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer("hi" , remove_nuktas = True )
        line = clean_text(line)
        tokens = list()
        for t in indic_tokenize.trivial_tokenize(line):
            tokens.append(t)
        line = tokens
        line = [ word.lower() for word in line]
        line = [word for word in line if not re.search(r'\d', word)]
        line = ' '.join(line)
        line = '<start> ' + line +' <end>'
        clean_hin_lines.append(line)
        
    return clean_hin_lines   


def to_pairs(text):
    lines = text.strip().split('\n')
    
    eng_hin_pairs = [line.split('\t') for line in lines]
    eng_hin_pairs = np.delete(eng_hin_pairs, [0,1,2,3,4] , axis = 1)
    eng_hin_pairs = eng_hin_pairs.tolist()
    clean_data = list()
    for pair in eng_hin_pairs:
        temp_data = list()
        for i, sentence in enumerate(pair):
            if i == 0:
                x = clean_english_data(sentence)
                temp_data.append(x)
            if i == 1:
                y = clean_hindi_data(sentence)
                temp_data.append(y)
        clean_data.append(temp_data)
    return eng_hin_pairs, clean_data

doc = load_doc("hindi-visual-genome-train.txt")

pairs , clean_data = to_pairs(doc)


In [3]:
# shuffling values
import random
BUFFER_SIZE = 28930
BATCH_SIZE = 64
num_examples = 20000
# train_dataset = clean_data[:n_train]
# val_dataset = clean_data[n_train:]
# # Let's limit the #training examples for faster training

# train_dataset = random.shuffle(train_dataset)
# val_dataset = random.shuffle(val_dataset)
# type(clean_data[0][1])

In [4]:
# train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [5]:
clean_hindi_lines = []
clean_english_lines = []

for i in range(len(clean_data)):
    clean_hindi_lines.append(clean_data[i][1])
    clean_english_lines.append(clean_data[i][0])
    


In [6]:
clean_hindi = list()
for i in range(len(clean_hindi_lines)):
    clean_hindi.append(clean_hindi_lines[i][0])
    
clean_english = list()
for i in range(len(clean_english_lines)):
    clean_english.append(clean_english_lines[i][0])


In [7]:
clean_english

['<start> it is an indoor scene <end>',
 '<start> computer screens turned on <end>',
 '<start> man has short hair <end>',
 '<start> photo album open on an adults lap <end>',
 '<start> there is a group of girls beside the black car <end>',
 '<start> child in a stroller <end>',
 '<start> tall metal lightpost <end>',
 '<start> wall is painted white <end>',
 '<start> there are several pictures on the wall <end>',
 '<start> woman facing the ocean <end>',
 '<start> this is an office layout <end>',
 '<start> four metallic chairs <end>',
 '<start> clutter is on a table <end>',
 '<start> a white microwave oven <end>',
 '<start> white suv driving through intersection <end>',
 '<start> person crossing street with umbrella <end>',
 '<start> man in gray pants leaning on building <end>',
 '<start> window on the building <end>',
 '<start> a man standing in between cars <end>',
 '<start> painting hanging on wall <end>',
 '<start> leg of the chair <end>',
 '<start> window of a boat <end>',
 '<start> th

In [8]:
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(padded_tensor_hin, padded_tensor_eng, test_size=0.2, random_state = 42)
input_train , input_val, target_train, target_val = train_test_split(clean_hindi , clean_english, test_size = 0.2, random_state = 42)

In [9]:
hin_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
hin_lang_tokenizer.fit_on_texts(clean_hindi)
hin_dict = hin_lang_tokenizer.word_index
tensor_hin = hin_lang_tokenizer.texts_to_sequences(input_train) 
padded_tensor_hin = tf.keras.preprocessing.sequence.pad_sequences(tensor_hin, padding='post')
# print(tensor)
# len(hin_dict)
input_tensor_train =  padded_tensor_hin
print(len(padded_tensor_hin))
print(len(hin_dict))

23144
7013


In [10]:
eng_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
eng_lang_tokenizer.fit_on_texts(clean_english)
eng_dict = eng_lang_tokenizer.word_index
tensor_eng = eng_lang_tokenizer.texts_to_sequences(target_train) 
padded_tensor_eng = tf.keras.preprocessing.sequence.pad_sequences(tensor_eng, padding='post')
target_tensor_train = padded_tensor_eng
# print(tensor)
# len(hin_dict)
print(len(padded_tensor_eng))
print(len(eng_dict))

23144
5553


In [11]:
# train test split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(padded_tensor_hin, padded_tensor_eng, test_size=0.2, random_state = 42)


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
# val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [13]:
train_dataset

<BatchDataset shapes: ((64, 34), (64, 26)), types: (tf.int32, tf.int32)>

In [14]:
len(target_tensor_train[0])

26

In [15]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 34]), TensorShape([64, 26]))

In [16]:
# some important parameters
vocab_inp_size = len(hin_lang_tokenizer.word_index)+1
vocab_tar_size = len(eng_lang_tokenizer.word_index)+1
max_length_input = len(input_tensor_train[0])
max_length_output = len(target_tensor_train[0])

embedding_dim = 100
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE

In [17]:
print("max_length_hindi, max_length_english, vocab_size_hindi, vocab_size_english")
max_length_input, max_length_output, vocab_inp_size, vocab_tar_size

max_length_hindi, max_length_english, vocab_size_hindi, vocab_size_english


(34, 26, 7014, 5554)

In [18]:
### english embedding:
embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt' ,  encoding = 'utf-8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:] , dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [19]:
num_words = min(vocab_tar_size, vocab_tar_size )
embedding_matrix = zeros((vocab_tar_size, 100))
for word, index in eng_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [20]:
print(vocab_tar_size)
embedding_matrix.shape
# embedding_matrix[90]

5554


(5554, 100)

In [21]:
### hindi embeddings

embeddings_dictionary_h = dict()
glove_file = open('hi-d100-glove.txt' ,  encoding = 'UTF-8', errors='ignore')
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:] , dtype='float32')
    embeddings_dictionary_h[word] = vector_dimensions

glove_file.close()

In [22]:
print(len(embeddings_dictionary_h))

1732951


In [23]:
num_words_h = vocab_inp_size
embedding_matrix_h = zeros((num_words_h, 100))
for word, index in hin_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary_h.get(word)
    if embedding_vector is not None:
        embedding_matrix_h[index] = embedding_vector

In [24]:
# embedding_matrix_h[46]

print(vocab_inp_size)
print(embedding_matrix_h.shape)

7014
(7014, 100)


In [25]:
class Encoder(tf.keras.Model):
    
    
    def __init__(self,embedding_matrix_h, vocab_size, embedding_dim, enc_units, batch_sz):
        
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        embedding_matrix = embedding_matrix_h
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [embedding_matrix_h])

        ##-------- LSTM layer in Encoder ------- ##
        self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')



    def call(self, x, hidden):
        
        x = self.embedding(x)
        output, h, c = self.lstm_layer(x, initial_state = hidden)
        return output, h, c

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [26]:
## Test Encoder Stack

encoder = Encoder(embedding_matrix_h, vocab_inp_size, embedding_dim, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (64, 34, 1024)
Encoder h vecotr shape: (batch size, units) (64, 1024)
Encoder c vector shape: (batch size, units) (64, 1024)


In [27]:
class Decoder(tf.keras.Model):
    def __init__(self, embedding_matrix, vocab_size, embedding_dim, dec_units, batch_sz, attention_type = 'luong'):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.attention_type = attention_type
        embedding_matrix = embedding_matrix
        
        #Embedding layer
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [embedding_matrix])
        
        #final dense layer on which softmax will be applied
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # define the fundamental cell for decoder recurrent structure
        
        self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
        
        # sampler 
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        # create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(self.dec_units, None, self.batch_sz*[max_length_input], self.attention_type)
        
        # wrap attention mechanism with the fundamental rnn cell
        self.rnn_cell = self.build_rnn_cell(batch_sz)
        
        # define the decoder with respect to fundamental rnn cell
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

        
        
    def build_rnn_cell(self, batch_sz):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                      self.attention_mechanism, attention_layer_size=self.dec_units)
        return rnn_cell
    
    def build_attention_mechanism(self, dec_units, memory,memory_sequence_length, attention_type = 'luong'):
         # ------------- #
        # typ: Which sort of attention (Bahdanau, Luong)
        # dec_units: final dimension of attention outputs 
        # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
        # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)
        if(attention_type == 'bahdanau'):
            return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
        
        else:
             return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
            
    def build_initial_state(self, batch_sz, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
        return decoder_initial_state


    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
        return outputs        

        
    

In [28]:
# Test decoder stack

decoder = Decoder(embedding_matrix, vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)

Decoder Outputs Shape:  (64, 25, 5554)


In [29]:
# defing optimizer and loss function

optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
    # real shape = (BATCH_SIZE, max_length_output)
    # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
    cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [30]:

checkpoint_dir = './training_checkpoints_h'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [31]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


        dec_input = targ[ : , :-1 ] # Ignore <end> token
        real = targ[ : , 1: ]         # ignore <start> token

        # Set the AttentionMechanism object with encoder_outputs
        decoder.attention_mechanism.setup_memory(enc_output)

        # Create AttentionWrapperState as initial_state for decoder
        decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
        pred = decoder(dec_input, decoder_initial_state)
        logits = pred.rnn_output
        loss = loss_function(real, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    # print(enc_hidden[0].shape, enc_hidden[1].shape)
    
    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
   # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [32]:
def evaluate_sentence(sentence):
    sentence = clean_hindi_data(sentence)
    string = " ".join(sentence)

    inputs = [hin_lang_tokenizer.word_index[i] for i in string.strip().split(" ")]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''
    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)
    dec_h = enc_h
    dec_c = enc_c
    start_tokens = tf.fill([inference_batch_size], eng_lang_tokenizer.word_index['<start>'])
    end_token = eng_lang_tokenizer.word_index['<end>']
    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
    # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
    # Setup Memory in decoder stack
    decoder.attention_mechanism.setup_memory(enc_out)

    # set decoder_initial_state
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


    ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
    ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
    ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function
    decoder_embedding_matrix = decoder.embedding.variables[0]
    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
    return outputs.sample_id.numpy()

def translate(sentence):
    result = evaluate_sentence(sentence)
    print(result)
    result = eng_lang_tokenizer.sequences_to_texts(result)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    return result, sentence

In [33]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [34]:
input_val

['<start> यह आकाश है <end>',
 '<start> बैंगनी और सफेद जूते के साथ लड़का <end>',
 '<start> डैपर लाल टोपी पहने हुए आदमी <end>',
 '<start> हरे रंग के पत्ते <end>',
 '<start> एक धावक पहले बेस से आगे निकल गया <end>',
 '<start> एक लाल रंग की ट्रेन <end>',
 '<start> अंधेरे लाल इमारत पर विचलित <end>',
 '<start> आदमी पर काला सूट जैकेट <end>',
 '<start> बड़ा धूसर भारतीय हाथी <end>',
 '<start> आदमी के पास एक अंगूठी है <end>',
 '<start> लड़की ने हार पहना हुआ है <end>',
 '<start> बोरहोल से पानी खींचता आदमी <end>',
 '<start> सड़क संकेत <end>',
 '<start> इमारत पर खिडकि <end>',
 '<start> मैदान में बैंगनी फूलों के शीर्ष <end>',
 '<start> फोन पकड़े हुए एक महीला <end>',
 '<start> सफेद प्लग के साथ काले आइपॉड <end>',
 '<start> ज़ेबरा पर धारियाँ <end>',
 '<start> काले जैतून का एक कटा हुआ टुकड़ा <end>',
 '<start> गाय लेटी हुई थी <end>',
 '<start> लकड़ी के स्लेट के साथ लकड़ी की बेंच <end>',
 '<start> यह एक इमारत है <end>',
 '<start> महिलाएं कुर्सी पर बैठी हैं <end>',
 '<start> प्लेट पर कटा हुआ लाल टमाटर <end>'

In [35]:
target_val

['<start> this is the sky <end>',
 '<start> boy with purple and white shoes <end>',
 '<start> man wearing dapper red hat <end>',
 '<start> green colored leaves <end>',
 '<start> a runner leading off of first base <end>',
 '<start> a red train <end>',
 '<start> windown on dark red building <end>',
 '<start> black suit jacket on man <end>',
 '<start> big grey indian elephant <end>',
 '<start> man has a ring on <end>',
 '<start> the girl is wearing a necklace <end>',
 '<start> man drawing water from borehole <end>',
 '<start> street sign <end>',
 '<start> a window on the building <end>',
 '<start> tops of purple flowers in the field <end>',
 '<start> woman holding a cell phone <end>',
 '<start> black ipod with white plug <end>',
 '<start> stripes on the zebra <end>',
 '<start> a sliced piece of black olive <end>',
 '<start> the cow lying down <end>',
 '<start> wooden bench with wooden slats <end>',
 '<start> this is a buiding <end>',
 '<start> women are sitting on the chair <end>',
 '<sta

In [36]:
# removing <start> and <end> from input_val
input_val_rem = []
for i in range(len(input_val)):
    l = input_val[i].split(" ")
    l.pop(0)
    n = len(l)-1
    l.pop(n)
    s = " ".join(l)
    input_val_rem.append(s)
    

In [37]:
input_val_rem.index('')

1346

In [38]:
print(len(input_val_rem), len(target_val))

5786 5786


In [39]:
target_val[1346]

'<start> tabby cat resting under desk <end>'

In [40]:
input_val_rem.index('')

1346

In [41]:
print(input_val_rem[1346] , target_val[1346])

 <start> tabby cat resting under desk <end>


In [42]:
# removing all the null values
input_val_rem.pop(1346)
target_val.pop(1346)

'<start> tabby cat resting under desk <end>'

In [43]:
print(len(input_val_rem), len(target_val))

5785 5785


In [44]:
input_val_rem.index('')

1655

In [45]:
target_val[1655]

'<start> tall giraffe standing in dirt <end>'

In [46]:
input_val_rem.pop(1655)
target_val.pop(1655)

'<start> tall giraffe standing in dirt <end>'

In [47]:
print(len(input_val_rem) , len(target_val))

5784 5784


In [48]:
input_val_rem.index('')

4508

In [49]:
input_val_rem.pop(4508)
target_val.pop(4508)

'<start> the ramp says supreme <end>'

In [50]:
print(len(input_val_rem) , len(target_val))

5783 5783


In [51]:
input_val_rem.pop(258)
target_val.pop(258)
print(len(input_val_rem) , len(target_val))

5782 5782


In [52]:
input_val_rem[258] , target_val[258]

('एक बहुत बड़ा टॉवर', '<start> a very large tower <end>')

In [53]:
translation = []
for i in range(0,100):
    translated_sent = translate(input_val_rem[i])
    translation.append(translated_sent)

[[17  8  5 27  3]]
Input: यह आकाश है
Predicted translation: ['this is the sky <end>']
[[ 61  14 202  12  10 269   3]]
Input: बैंगनी और सफेद जूते के साथ लड़का
Predicted translation: ['boy with purple and white shoes <end>']
[[11 23 15 93  3]]
Input: डैपर लाल टोपी पहने हुए आदमी
Predicted translation: ['man wearing red hat <end>']
[[ 19 213   6   5  19   3]]
Input: हरे रंग के पत्ते
Predicted translation: ['green leaves on the green <end>']
[[  4  11 484 189   7   4 214   3]]
Input: एक धावक पहले बेस से आगे निकल गया
Predicted translation: ['a man coming out of a ski <end>']
[[ 4 15 26  3]]
Input: एक लाल रंग की ट्रेन
Predicted translation: ['a red train <end>']
[[  5 360 784   6   4  15  31   3]]
Input: अंधेरे लाल इमारत पर विचलित
Predicted translation: ['the sun mounted on a red building <end>']
[[ 13 184   6  11   3]]
Input: आदमी पर काला सूट जैकेट
Predicted translation: ['black jacket on man <end>']
[[ 37 107  75  75   3]]
Input: बड़ा धूसर भारतीय हाथी
Predicted translation: ['large grey ele

[[ 11 345 120   3]]
Input: सर्फबोर्ड ले जाने वाला आदमी
Predicted translation: ['man carrying surfboard <end>']
[[ 68   6   5  63   7   5 268   3]]
Input: पहाड़ की चोटी पर बर्फ
Predicted translation: ['snow on the top of the mountain <end>']
[[ 83   9   5 939   3]]
Input: शॉट में पक्षी
Predicted translation: ['bird in the shot <end>']
[[17  8  4 11  3]]
Input: यह एक आदमी है
Predicted translation: ['this is a man <end>']
[[75 35  9 75  3]]
Input: हाथी समाशोधन में खड़ा है
Predicted translation: ['elephant standing in elephant <end>']
[[   5 3086    6    5   42    3]]
Input: घड़ी पर यूडी
Predicted translation: ['the lifeguard on the clock <end>']
[[  4  18 133   4  78   3]]
Input: पिज्जा का एक टुकड़ा खा रही महिला
Predicted translation: ['a woman eating a pizza <end>']
[[ 4 80 35  9  5 57  3]]
Input: सर्विसमैन एक लाइन में खड़ा है
Predicted translation: ['a zebra standing in the side <end>']
[[ 5 39  7  4 21  3]]
Input: एक व्यक्ति का सिर
Predicted translation: ['the head of a person <end>']


In [56]:
len(translation) 

100

In [55]:
target_val_1 = target_val[:100]
len(target_val_1)

100

In [ ]:
# 1-gram individual BLEU
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'small', 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
print(score)

In [ ]:
candidate

In [57]:

for i in range(len(target_val_1)):
    target_val_1[i] = target_val_1[i].split(" ")[1:-1]

In [58]:
target_val_1

[['this', 'is', 'the', 'sky'],
 ['boy', 'with', 'purple', 'and', 'white', 'shoes'],
 ['man', 'wearing', 'dapper', 'red', 'hat'],
 ['green', 'colored', 'leaves'],
 ['a', 'runner', 'leading', 'off', 'of', 'first', 'base'],
 ['a', 'red', 'train'],
 ['windown', 'on', 'dark', 'red', 'building'],
 ['black', 'suit', 'jacket', 'on', 'man'],
 ['big', 'grey', 'indian', 'elephant'],
 ['man', 'has', 'a', 'ring', 'on'],
 ['the', 'girl', 'is', 'wearing', 'a', 'necklace'],
 ['man', 'drawing', 'water', 'from', 'borehole'],
 ['street', 'sign'],
 ['a', 'window', 'on', 'the', 'building'],
 ['tops', 'of', 'purple', 'flowers', 'in', 'the', 'field'],
 ['woman', 'holding', 'a', 'cell', 'phone'],
 ['black', 'ipod', 'with', 'white', 'plug'],
 ['stripes', 'on', 'the', 'zebra'],
 ['a', 'sliced', 'piece', 'of', 'black', 'olive'],
 ['the', 'cow', 'lying', 'down'],
 ['wooden', 'bench', 'with', 'wooden', 'slats'],
 ['this', 'is', 'a', 'buiding'],
 ['women', 'are', 'sitting', 'on', 'the', 'chair'],
 ['sliced', 'red',

In [59]:
t = []
for i in range(len(translation)):
    t.append(translation[i][0])    

In [60]:
t1=[]
for i in range(len(t)):
    l = t[i][0].split(" ")[:-1]
    s = " ".join(l)
    t1.append(s)

t1

['this is the sky',
 'boy with purple and white shoes',
 'man wearing red hat',
 'green leaves on the green',
 'a man coming out of a ski',
 'a red train',
 'the sun mounted on a red building',
 'black jacket on man',
 'large grey elephant elephant',
 'the man has a ring',
 'the girl is wearing a necklace',
 'man drinking water from the water',
 'the road sign',
 'window on the building',
 'purple dome on field',
 'a woman holding a woman',
 'black tape with white covering background',
 'stripes on the zebra',
 'a sliced of black olive',
 'cow is lying on the cow',
 'wooden bench with wood leaf',
 'this is a building',
 'the ladies rest sitting on the chair',
 'red tomato on plate',
 'a cows standing on the two round',
 'this is a toilet',
 'the hydrant is yellow',
 'people sitting on a table',
 'blue surf board',
 'the metal trunk of the giraffe',
 'a shallow body of water',
 'a group of people',
 'phone on the table',
 'glass window on the boat',
 'train on the train',
 'a ibis on a 

In [61]:
# # 1-gram individual BLEU
# from nltk.translate.bleu_score import sentence_bleu
# reference = [['this', 'is', 'small', 'test']]
# candidate = ['this', 'is', 'a', 'test']
# score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
# print(score)

0.75


C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [62]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
avg_1g_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    print(score)
    avg_1g_score = avg_1g_score + score

avg_1g_score = avg_1g_score/len(t1)

1.0
1.0
0.7788007830714049
0.4
0.2857142857142857
1.0
0.42857142857142855
0.7788007830714049
0.5
0.8
1.0
0.5
0.3333333333333333
0.7788007830714049
0.23618327637050734
0.6
0.5
1.0
0.8187307530779819
0.5
0.6
0.75
0.5714285714285714
0.5841005873035536
0.42857142857142855
0.7788007830714049
1.0
0.8
1.0
0.3333333333333333
0.8
1.0
0.75
0.8
0.5
0.8
0.8
1.0
0.5
0.3274923012311928
1.0
0.5
0.7165313105737893
0.4
0.49123845184678916
0.8
1.0
0.8
1.0
0.8571428571428571
0.3333333333333333
1.0
0.5
0.6
1.0
1.0
1.0
0.2857142857142857
0.3333333333333333
1.0
1.0
1.0
0.8
0.75
0.5841005873035536
0.5841005873035536
1.0
0.5
0.8
0.8333333333333334
1.0
0.6
1.0
1.0
0.6065306597126334
0.7165313105737893
0.7142857142857143
0.75
1.0
0.75
0.6
0.6549846024623855
0.5
0.8
0.7054014374088451
1.0
1.0
1.0
1.0
0.6
0.2727272727272727
0.7054014374088451
0.42857142857142855
1.0
0.8
0.375
0.16666666666666669
1.0
0.7142857142857143
1.0


C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [63]:
avg_1g_score

0.7229187675520933

In [64]:
avg_2g_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(0, 1, 0, 0))
    print(score)
    avg_2g_score = avg_2g_score + score

avg_2g_score = avg_2g_score/len(t1)

1.0
1.0
0.5192005220476033
2.2250738585072626e-308
2.2250738585072626e-308
1.0
0.16666666666666669
0.5192005220476033
2.2250738585072626e-308
0.75
1.0
0.2
2.2250738585072626e-308
0.7788007830714049
1.051050468137224e-308
0.5
0.2
1.0
0.6140480648084865
0.2
0.5
0.6666666666666666
0.5
0.2596002610238016
0.16666666666666669
0.7788007830714049
1.0
0.5
1.0
2.2250738585072626e-308
0.75
1.0
0.3333333333333333
0.5
2.2250738585072626e-308
0.5
0.5
1.0
0.4
1.8217363958297824e-308
1.0
2.2250738585072626e-308
0.35826565528689464
0.25
0.40936537653899097
0.5
1.0
0.25
1.0
0.6666666666666666
0.2
1.0
0.2
0.25
1.0
1.0
1.0
2.2250738585072626e-308
2.2250738585072626e-308
1.0
1.0
1.0
0.75
0.3333333333333333
1.7328892633971685e-308
0.5192005220476033
1.0
2.2250738585072626e-308
0.5
0.4
1.0
2.2250738585072626e-308
1.0
1.0
0.6065306597126334
0.35826565528689464
0.3333333333333333
0.6666666666666666
1.0
0.6666666666666666
0.5
0.40936537653899097
0.2
0.75
0.16929634497812282
1.0
1.0
1.0
1.0
0.5
0.100000000000000

In [65]:
avg_2g_score 

0.5446472369260715

In [66]:
avg_ng_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
    print(score)
    avg_ng_score = avg_ng_score + score

avg_ng_score = avg_ng_score/len(t1)

1.0
1.0
1.0497255803085492e-154
1.4488496539373276e-231
1.331960397810445e-231
1.2213386697554703e-77
7.711523862191631e-155
7.227401369829121e-78
1.5319719891192393e-231
0.668740304976422
1.0
8.38826642100846e-155
1.384292958842266e-231
0.7788007830714049
7.236523273960503e-232
6.86809206056511e-78
8.38826642100846e-155
1.0
7.070696784820904e-78
8.38826642100846e-155
6.86809206056511e-78
8.636168555094496e-78
0.41113361690051975
8.214546595247418e-155
7.711523862191631e-155
0.7788007830714049
1.0
1.1862800137389335e-154
1.2213386697554703e-77
1.384292958842266e-231
0.668740304976422
1.0
1.0547686614863434e-154
7.380245217279165e-78
1.5319719891192393e-231
7.380245217279165e-78
7.380245217279165e-78
1.0
5.775353993361614e-78
1.1862177682648818e-231
1.0
1.5319719891192393e-231
8.987727354491445e-155
8.38826642100846e-155
5.623118184955381e-78
7.380245217279165e-78
1.0
9.975386110402101e-155
1.0
8.44484326442819e-78
7.57965434483665e-155
1.0
8.38826642100846e-155
9.283142785759642e-155
1

In [67]:
avg_ng_score

0.3264369640294902

In [ ]:
len(t1)

In [ ]:
target_val_1[2996] , t1[2996]